In [1]:
#place your eternafold params
#see https://github.com/eternagame/EternaFold for more details
%env ETERNAFOLD_PATH=/home/hoyso/miniconda3/envs/hoyso_ml/bin/eternafold-bin
%env ETERNAFOLD_PARAMETERS=/home/hoyso/miniconda3/envs/hoyso_ml/lib/eternafold-lib/parameters/EternaFoldParams.v1

env: ETERNAFOLD_PATH=/home/hoyso/miniconda3/envs/hoyso_ml/bin/eternafold-bin
env: ETERNAFOLD_PARAMETERS=/home/hoyso/miniconda3/envs/hoyso_ml/lib/eternafold-lib/parameters/EternaFoldParams.v1


In [9]:
from arnie.mfe import mfe
import pandas as pd
from tqdm.autonotebook import tqdm

/tmp/ipykernel_713162/3231773077.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
import shutil
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count

def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks
    
def mfe_eternafold(seqs):
    result = []
    for x in tqdm(seqs):
        result.append(mfe(x,package="eternafold"))
    return result

In [12]:
train_df = pd.read_parquet('train_data_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(mfe_eternafold)(x)
    for i,x in enumerate(split_dataframe(train_df.sequence, 10000))
)
train_df['mfe_eternafold'] = np.concatenate(result, axis=0)
train_df.to_parquet('train_data_new.parquet')
del train_df

  0%|          | 36/10000 [00:02<12:39, 13.12it/s]


  0%|          | 40/10000 [00:02<11:36, 14.30it/s]

  0%|          | 39/10000 [00:02<11:54, 13.95it/s]

  0%|          | 43/10000 [00:02<10:49, 15.33it/s]

  0%|          | 43/10000 [00:02<10:58, 15.12it/s]

  0%|          | 6/10000 [00:00<09:57, 16.73it/s]

KeyboardInterrupt: 

In [14]:
test_df = pd.read_parquet('test_sequences_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(mfe_eternafold)(x)
    for i,x in enumerate(split_dataframe(test_df.sequence, 10000))
)
test_df['mfe_eternafold'] = np.concatenate(result, axis=0)
test_df.to_parquet('test_sequences_new.parquet')
del test_df

  1%|          | 76/10000 [00:03<08:25, 19.62it/s]


  1%|          | 70/10000 [00:03<09:16, 17.83it/s]

  1%|          | 54/10000 [00:03<12:05, 13.70it/s]


  1%|          | 57/10000 [00:04<11:38, 14.24it/s]


  1%|          | 70/10000 [00:03<09:13, 17.94it/s]

  0%|          | 2/10000 [00:00<12:44, 13.08it/s]

KeyboardInterrupt: 